In [ ]:
# 실습용 데이터셋 - 서울시 공공자전거 데이터
# %pip install opendata-kr -q

In [90]:
from IPython.display import Image
import numpy as np
import pandas as pd
import seaborn as sns
from opendata import dataset

# import warnings
# warnings.filterwarnings('ignore')

# e notation 표현 방식 변경
pd.options.display.float_format = '{: .2f}'.format

# 모든 컬럼 표시
pd.set_option('display.max_columns', None)


In [ ]:
'''데이터 프레임의 복사와 결측치 처리'''
# 복사 copy() : 데이터프레임 복제/ 원본영향 없음
# 결측치 - 결측 데이터 유무/ 결측 데이터 채우기 / 결측 데이터 제거하기
## 결측치 확인 - isnull(), isnan(),notnull()
## 결측 데이터 필터링 ; loc
## 결측 데이터 채우기 : fillna(채울값)
## 결측 데이터 제거하기 : dropna(how = 'any') * how = 'any' ; 1개라도 NaN 값이 있는 행 제거 (기본설정)

'''데이터 전처리, 추가, 삭제, 변환'''
# 컬럼 추가
df1 = df.copy()

df1['VIP'] = True # 마지막에 새로운 컬럼 추가
df1.insert(5, 'RICH', df1['fare'] > 100) # 중간에 새로운 컬럼 추가:insert(컬럼인덱스,컬럼명, 값)

# 삭제

## row 삭제 ; drop(인덱스번호/범위/팬시인덱싱)
df1.drop(1) # 인덱스 지정하여 삭제
df1.drop(np.arange(10)) # 행범위 지정하여 삭제
df1.drop([1,3,5,7,9])

## column 삭제 ; drop([컬럼이름들],axis=1)
df1.drop('class', axis=1)
df1.drop(['RICH','VIP'], axis=1).head()
df1.drop(['RICH','VIP'], axis=1, inplace=True) # inplace = True : 삭제 내역을 데이터프레임에 바로 반영

# 컬럼 간 연산
df1 = df.copy()

## 합산
df1['Family'] = df1['sibsp'] + df1['parch']
df1.insert(1,'FA', df1['sibsp'] + df1['parch'])

## 문자열의 합(이어 붙이기)
df1['gender'] = df1['who'] + '-' + df1['sex']
# df1.insert(0,'GE',df1['who'] + '-' + df1['sex'])


## 소숫점 반올림 : round(숫자, 소수점자리) - 컬럼이 NaN 값을 하나라도 포함하고 있다면 연산 결과는 NaN 
df1['round'] = round(df1['fare']/df['age'], 2)
# df1.insert(0,'round',round(df1['fare']/df['age'], 2))

df1.loc[df1['age'].isnull(), 'deck':].head  # NaN 환산

# category type & 변경
# df1.info()
df1['who'].info() # 데이터타입 확인
df1['who'].astype('category').tail() # 타입 변경이 적용된 것은 아님
df1['who'] = df['who'].astype('category') # 데이터타입 변경 적용 

df1['who'].cat.categories # type이 category 이면, ".cat"로 접근하여 category 타입이 제공하는 속성 사용 가능
df1['who'] = df1['who'].cat.rename_categories(['아이', '남자', '여자']) ## 카테고리 이름 변경 / 변경 시 원래 순서에 맞게 입력
## df1['who'].cat.rename_categories = ['아이', '남자', '여자'] => not working
df1['who'].value_counts()

# datatime type & 변경
# 데이터 타입 변경 pd.to_datetime(df2['컬럼이름'])
df2['대여일자'] = pd.to_datetime(df2['대여일자']) # 대여일자의 데이터 타입을 데이트 타임으로 변경
df2['대여일자'].dt.year
df2['대여일자'].dt.month
df2['대여일자'].dt.dayofweek

# 구간 나누기(binning); pd.cut(): continuous value를 구간으로 나누어 카테고리화
## 직접 구간 설정
bins = [0,6000,100000, df2['이동거리'].max()] 
pd.cut(df2['이동거리'], bins, right=False) # right = False ; 우측 범위 불포함
pd.cut(df2['이동거리'], bins, labels=['적음','보통','많음'], right=False) # 라벨 지정 ; 지정한 bin의 갯수보다 1개 적어야 함

## 구간 갯수만 설정 ; 구간 범위 자동
df2['이동거리_cut'] = pd.cut(df2['이동거리'], bins=3) # bins = 구간의 갯수 지정, 범위는 자동
df2['이동거리_cut'].value_counts() # => 데이터의 솔림 현상 심함

## 동일한 갯수 갖도록 구간 분할 : pd.qcut()
df2['이동거리_qcut'] = pd.qcut(df2['이동거리'], q=3)
df2['이동거리_qcut'].value_counts()

qcut_bins = [0, 0.2, 0.8, 1] # 구간 범위 설정도 가능
pd.qcut(df2['이동거리'], qcut_bins)
pd.qcut(df2['이동거리'], qcut_bins, labels=['적음', '보통', '많음']).value_counts()  # 라벨링

''' 데이터셋 가져오기 '''
df = sns.load_dataset('titanic')
df.head()



<class 'pandas.core.series.Series'>
RangeIndex: 891 entries, 0 to 890
Series name: who
Non-Null Count  Dtype 
--------------  ----- 
891 non-null    object
dtypes: object(1)
memory usage: 7.1+ KB


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.00,1,0,7.25,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.00,1,0,71.28,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.00,0,0,7.92,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.00,1,0,53.10,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.00,0,0,8.05,S,Third,man,True,NaN,Southampton,no,True


In [5]:
# 데이터 복사
df_copy = df.copy()
df_copy

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.00,1,0,7.25,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.00,1,0,71.28,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.00,0,0,7.92,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.00,1,0,53.10,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.00,0,0,8.05,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.00,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.00,0,0,30.00,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.00,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True


In [ ]:
'''연습문제
타이타닉 승객 데이터에서 age가 결측치인 승객의 나이를 30세로 일괄 채우기'''
df_copy['age'].isnull().sum()          # age가 null값인 데이터 갯수 확인
df_copy.loc[df_copy['age'].isnull(),:] # age가 null값인 데이터 모두 확인

df_copy['age'] = df_copy['age'].fillna(30)



In [ ]:
# 검증코드
# 본 Cell을 실행시 ERROR가 발생하지 않아야 함
assert df_copy['age'].isnull().sum() == 0 # assert condition, optional message
assert df_copy['age'].mean().round(4) == 29.7589


In [46]:
'''연습문제
타이타닉 승객 데이터에서 남자 승객의 나이 결측치는, 남자 승객의 평균 나이로 채웁니다.
여자 승객의 나이 결측치는, 여자 승객의 평균 나이로 채웁니다'''

df_copy1 = df.copy()

df_copy1.loc[(df_copy1['age'].isnull()) & (df_copy1['sex'] == 'female'), :]['sex'].count()
df_copy1.loc[(df_copy1['age'].isnull()) & (df_copy1['sex'] == 'male'), :]['sex'].count()

##
female = df_copy1['sex'] == 'female'
female_avg = df_copy1.loc[female,'age'].mean()

male = df_copy1['sex'] == 'male'
male_avg = df_copy1.loc[male,'age'].mean()


df_copy1.loc[female,'age'] = df_copy1.loc[female,'age'].fillna(female_avg)
df_copy1.loc[male,'age'] = df_copy1.loc[male,'age'].fillna(male_avg)



In [48]:
# 검증코드
# Cell 실행시 오류가 발생하지 않으면 PASS
assert (df_copy1['age'].isnull().sum() == 0)
assert df_copy1['age'].mean().round(5) == 29.73603


In [ ]:
'''데이터 전처리, 추가, 삭제, 변환'''

# 컬럼 추가
df1 = df.copy()

df1['VIP'] = True # 마지막에 새로운 컬럼 추가
df1.insert(5, 'RICH', df1['fare'] > 100) # 중간에 새로운 컬럼 추가:insert(컬럼인덱스,컬럼명, 값)

# 삭제

## row 삭제 ; drop(인덱스번호/범위/팬시인덱싱)
df1.drop(1) # 인덱스 지정하여 삭제
df1.drop(np.arange(10)) # 행범위 지정하여 삭제
df1.drop([1,3,5,7,9])

## column 삭제 ; drop([컬럼이름들],axis=1)
df1.drop('class', axis=1)
df1.drop(['RICH','VIP'], axis=1).head()
df1.drop(['RICH','VIP'], axis=1, inplace=True) # inplace = True : 삭제 내역을 데이터프레임에 바로 반영

# 컬럼 간 연산
df1 = df.copy()

## 합산
df1['Family'] = df1['sibsp'] + df1['parch']
df1.insert(1,'FA', df1['sibsp'] + df1['parch'])

## 문자열의 합(이어 붙이기)
df1['gender'] = df1['who'] + '-' + df1['sex']
# df1.insert(0,'GE',df1['who'] + '-' + df1['sex'])


## 소숫점 반올림 : round(숫자, 소수점자리) - 컬럼이 NaN 값을 하나라도 포함하고 있다면 연산 결과는 NaN 
df1['round'] = round(df1['fare']/df['age'], 2)
# df1.insert(0,'round',round(df1['fare']/df['age'], 2))

df1.loc[df1['age'].isnull(), 'deck':].head  # NaN 환산

# category type & 변경
# df1.info()
df1['who'].info() # 데이터타입 확인
df1['who'].astype('category').tail() # 타입 변경이 적용된 것은 아님
df1['who'] = df['who'].astype('category') # 데이터타입 변경 적용 

df1['who'].cat.categories # type이 category 이면, ".cat"로 접근하여 category 타입이 제공하는 속성 사용 가능
df1['who'] = df1['who'].cat.rename_categories(['아이', '남자', '여자']) ## 카테고리 이름 변경 / 변경 시 원래 순서에 맞게 입력
## df1['who'].cat.rename_categories = ['아이', '남자', '여자'] => not working
df1['who'].value_counts()


<class 'pandas.core.series.Series'>
RangeIndex: 891 entries, 0 to 890
Series name: who
Non-Null Count  Dtype 
--------------  ----- 
891 non-null    object
dtypes: object(1)
memory usage: 7.1+ KB


who
man      537
woman    271
child     83
Name: count, dtype: int64

In [ ]:
# 데이터 다운로드
dataset.download('서울시자전거')

# /Users/kwonkyoungmi/workspace_Python/data/서울시자전거/seoul_bicycle.csv

[서버] Jaen
======= 다운로드 시작 =======

data/서울시자전거/seoul_bicycle.csv


  0%|          | 0.00/31.1M [00:00<?, ?B/s]


======= 다운로드 완료 =======


In [96]:
# 데이터셋 로드
df2 = pd.read_csv('/Users/kwonkyoungmi/workspace_Python/data/서울시자전거/seoul_bicycle.csv')
df2.head()

,대여일자,대여소번호,대여소명,대여구분코드,성별,연령대코드,이용건수,운동량,탄소량,이동거리,이용시간
0,Jan-20-2020,3,중랑센터,일일(회원),M,AGE_003,3,61.82,0.52,2230.00,75
1,Jan-20-2020,3,중랑센터,일일(회원),M,AGE_004,1,39.62,0.28,1220.00,15
2,Jan-20-2020,3,중랑센터,정기,M,AGE_005,3,430.85,4.01,17270.00,53
3,Jan-20-2020,5,상암센터 정비실,일일(회원),\N,AGE_005,2,1.79,0.02,90.00,33
4,Jan-20-2020,5,상암센터 정비실,정기,F,AGE_003,1,4501.96,45.47,196010.00,64


In [ ]:
df2.info() # 대여일자의 data type 확인 ; object

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 327231 entries, 0 to 327230
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   대여일자    327231 non-null  object 
 1   대여소번호   327231 non-null  int64  
 2   대여소명    327231 non-null  object 
 3   대여구분코드  327231 non-null  object 
 4   성별      272841 non-null  object 
 5   연령대코드   327231 non-null  object 
 6   이용건수    327231 non-null  int64  
 7   운동량     327231 non-null  object 
 8   탄소량     327231 non-null  object 
 9   이동거리    327231 non-null  float64
 10  이용시간    327231 non-null  int64  
dtypes: float64(1), int64(3), object(7)
memory usage: 27.5+ MB


In [100]:
df2['대여일자'] = pd.to_datetime(df2['대여일자']) # 대여일자의 데이터 타입을 데이트 타임으로 변경
df2['대여일자'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 327231 entries, 0 to 327230
Series name: 대여일자
Non-Null Count   Dtype         
--------------   -----         
327231 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 2.5 MB


In [101]:
df2['대여일자'].dt.year

0         2020
1         2020
2         2020
3         2020
4         2020
          ... 
327226    2020
327227    2020
327228    2020
327229    2020
327230    2020
Name: 대여일자, Length: 327231, dtype: int32

In [102]:
df2['대여일자'].dt.month

0         1
1         1
2         1
3         1
4         1
         ..
327226    5
327227    5
327228    5
327229    5
327230    5
Name: 대여일자, Length: 327231, dtype: int32

In [103]:
df2['대여일자'].dt.dayofweek

0         0
1         0
2         0
3         0
4         0
         ..
327226    2
327227    2
327228    2
327229    2
327230    2
Name: 대여일자, Length: 327231, dtype: int32

In [105]:
df2.head()

,대여일자,대여소번호,대여소명,대여구분코드,성별,연령대코드,이용건수,운동량,탄소량,이동거리,이용시간
0,2020-01-20,3,중랑센터,일일(회원),M,AGE_003,3,61.82,0.52,2230.00,75
1,2020-01-20,3,중랑센터,일일(회원),M,AGE_004,1,39.62,0.28,1220.00,15
2,2020-01-20,3,중랑센터,정기,M,AGE_005,3,430.85,4.01,17270.00,53
3,2020-01-20,5,상암센터 정비실,일일(회원),\N,AGE_005,2,1.79,0.02,90.00,33
4,2020-01-20,5,상암센터 정비실,정기,F,AGE_003,1,4501.96,45.47,196010.00,64


In [ ]:
# 구간 나누기(binning); pd.cut(): continuous value를 구간으로 나누어 카테고리화
## 직접 구간 설정
bins = [0,6000,100000, df2['이동거리'].max()] 
pd.cut(df2['이동거리'], bins, right=False) # right = False ; 우측 범위 불포함
pd.cut(df2['이동거리'], bins, labels=['적음','보통','많음'], right=False) # 라벨 지정 ; 지정한 bin의 갯수보다 1개 적어야 함

## 구간 갯수만 설정 ; 구간 범위 자동
df2['이동거리_cut'] = pd.cut(df2['이동거리'], bins=3) # bins = 구간의 갯수 지정, 범위는 자동
df2['이동거리_cut'].value_counts() # => 데이터의 솔림 현상 심함

## 동일한 갯수 갖도록 구간 분할 : pd.qcut()
df2['이동거리_qcut'] = pd.qcut(df2['이동거리'], q=3)
df2['이동거리_qcut'].value_counts()

qcut_bins = [0, 0.2, 0.8, 1] # 구간 범위 설정도 가능
pd.qcut(df2['이동거리'], qcut_bins)
pd.qcut(df2['이동거리'], qcut_bins, labels=['적음', '보통', '많음']).value_counts()  # 라벨링


In [ ]:
bins = [0,6000,100000, df2['이동거리'].max()] # 직접 범위 설정

pd.cut(df2['이동거리'], bins, right=False) # right = False ; 우측 범위 불포함

pd.cut(df2['이동거리'], bins, labels=['적음','보통','많음'], right=False) # 라벨 지정 ; 지정한 bin의 갯수보다 1개 적어야 함

0         적음
1         적음
2         보통
3         적음
4         많음
          ..
327226    보통
327227    적음
327228    적음
327229    적음
327230    많음
Name: 이동거리, Length: 327231, dtype: category
Categories (3, object): ['적음' < '보통' < '많음']

In [ ]:
df2['이동거리_cut'] = pd.cut(df2['이동거리'], bins=3) # bins = 구간의 갯수 지정, 범위는 자동
df2['이동거리_cut'].value_counts() # => 데이터의 솔림 현상 심함

이동거리_cut
(-56709.053, 18903017.647]      327216
(18903017.647, 37806035.293]        12
(37806035.293, 56709052.94]          3
Name: count, dtype: int64

In [ ]:
# 동일한 갯수 갖도록 구간 분할 : pd.qcut()
df2['이동거리_qcut'] = pd.qcut(df2['이동거리'], q=3)
df2['이동거리_qcut'].value_counts()

qcut_bins = [0, 0.2, 0.8, 1]
pd.qcut(df2['이동거리'], qcut_bins)

pd.qcut(df2['이동거리'], qcut_bins, labels=['적음', '보통', '많음']).value_counts() 


이동거리
보통    196307
적음     65482
많음     65442
Name: count, dtype: int64

In [ ]:
'''연습문제
df3 에서 1,3,5 번행 삭제
df3 에서 embarked, class, alone 컬럼 삭제
df3의 상위 10개 행 출력'''

df3 = df.copy()

# df3 에서 1,3,5 번행 삭제
df3.drop([1,3,5], inplace= True)
# df3 에서 embarked, class, alone 컬럼 삭제
df3.drop(['embarked', 'class', 'alone'], axis = 1, inplace = True)
# df3의 상위 10개 행 출력
df3.head(10)

,survived,pclass,sex,age,sibsp,parch,fare,who,adult_male,deck,embark_town,alive
0,0,3,male,22.00,1,0,7.25,man,True,NaN,Southampton,no
2,1,3,female,26.00,0,0,7.92,woman,False,NaN,Southampton,yes
4,0,3,male,35.00,0,0,8.05,man,True,NaN,Southampton,no
6,0,1,male,54.00,0,0,51.86,man,True,E,Southampton,no
7,0,3,male,2.00,3,1,21.07,child,False,NaN,Southampton,no
8,1,3,female,27.00,0,2,11.13,woman,False,NaN,Southampton,yes
9,1,2,female,14.00,1,0,30.07,child,False,NaN,Cherbourg,yes
10,1,3,female,4.00,1,1,16.70,child,False,G,Southampton,yes
11,1,1,female,58.00,0,0,26.55,woman,False,C,Southampton,yes
12,0,3,male,20.00,0,0,8.05,man,True,NaN,Southampton,no


In [138]:
'''연습문제
Iris 붓꽃 데이터셋 활용

sepal 컬럼을 생성하고, sepal_length와 sepal_width 곱한값을 대입 => 상위 5개 행만 출력
petal 컬럼을 생성하고, petal_length와 petal_width 곱한값을 대입 => 상위 5개 행만 출력
petal_length, petal width 컬럼 제거 => 상위 5개 행만 출력
species가 setosa인 꽃 중 sepal을 기준으로 내림차순 정렬
sepal 컬럼의 평균과 petal 컬럼의 평균의 차이 구하기
'''

# iris = sns.load_dataset('iris')

iri = iris.copy()
iri.head()
iri.describe()
iri.info()

# sepal 컬럼을 생성하고, sepal_length와 sepal_width 곱한값을 대입 => 상위 5개 행만 출력
iri['sepal'] = iri['sepal_length'] * iri['sepal_width']
iri.iloc[:,3:].head()
# petal 컬럼을 생성하고, petal_length와 petal_width 곱한값을 대입 => 상위 5개 행만 출력
iri['petal'] = iri['petal_length'] * iri['petal_width']
iri.iloc[:,4:].head()
# petal_length, petal width 컬럼 제거 => 상위 5개 행만 출력
iri.drop(['petal_length','petal_width'], axis = 1, inplace = True)
iri.head()
# species가 setosa인 꽃 중 sepal을 기준으로 내림차순 정렬
iri[iri['species'] == 'setosa'].sort_values(by = 'sepal', ascending = False).head()
# sepal 컬럼의 평균과 petal 컬럼의 평균의 차이 구하기
iri['sepal'].mean() - iri['petal'].mean()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


12.028799999999997

In [ ]:
'''further practics

# age를 다음과 같은 구간을 가지도록 나누고, age_bin 컬럼 생성 후 값 대입 분포 출력 ; cut() 함수 활용
   1. 0세 초과 ~ 15세 이하
   2. 15세 초과 ~ 30세 이하
   3. 30세 초과 ~ 45세 이하
   4. 45세 초과 ~ 최대값
# 나이를 3개의 균등 분할을 갖도록 구간 나누고, 분포 출력 ; qcut() 함수 활용
   labels = ['young','normal','old']

# 생존자, 사망자 숫자 출력
# 남성 생존자, 여성 생존자 숫자, 비율 출력
   남성 생존자 비율 = (남성 생존자 숫자) /  (전체 생존자 숫자)
   여성 생존자 비율 = (여성 생존자 숫자) /  (전체 생존자 숫자)
# 객실 등급별 생존자 숫자, 비율
   1등급 객실 생존자 비율 = (1등급 객실 생존자 숫자) / (전체 생존자 숫자)
# 10대 미만 승객 숫자
# 10대 미만 생존자 숫자
# 10대 미만 승객이 생존자 배율
'''

titanic = df.copy()

In [144]:
#  age를 다음과 같은 구간을 가지도록 나누고, age_bin 컬럼 생성 후 값 대입 분포 출력 ; cut() 함수 활용
#    1. 0세 초과 ~ 15세 이하
#    2. 15세 초과 ~ 30세 이하
#    3. 30세 초과 ~ 45세 이하
#    4. 45세 초과 ~ 최대값

bins = [0,15,30,45,titanic['age'].max()]
titanic['age_bin'] = pd.cut(titanic['age'], bins)

titanic['age_bin'].value_counts()

age_bin
(15.0, 30.0]    326
(30.0, 45.0]    202
(45.0, 80.0]    103
(0.0, 15.0]      83
Name: count, dtype: int64

In [147]:
# 나이를 3개의 균등 분할을 갖도록 구간 나누고, 분포 출력 ; qcut() 함수 활용
labels = ['young','normal','old']

pd.qcut(titanic['age'], labels = labels, q=3).value_counts()

age
young     246
old       236
normal    232
Name: count, dtype: int64

In [165]:
titanic['survived'].unique()
titanic['survived'].value_counts()
titanic.survived.unique() # == titanic['survived'].unique()
titanic.survived.dtype 

dtype('int64')

In [208]:
titanic['pclass'].unique()

array([3, 1, 2])

In [227]:
# 생존자, 사망자 숫자 출력
titanic['survived'].value_counts().loc[1]
titanic[(titanic.survived == 1)].count().survived # == titanic[(titanic.survived == 1)].count()['survived']
                                                  # == len(titanic[ titanic['survived'] == 1])
num_alive = len(titanic[ titanic.survived == 1])
print(f'전체 생존자 수 : {num_alive:.2f}')

# 남성 생존자, 여성 생존자 숫자, 비율 출력
#    남성 생존자 비율 = (남성 생존자 숫자) /  (전체 생존자 숫자)
#    여성 생존자 비율 = (여성 생존자 숫자) /  (전체 생존자 숫자)

print(""); print('- 생존자 성비')
for gender in titanic['sex'].unique():
    alive = titanic[(titanic.survived == 1) & (titanic.sex == gender)]['survived'].count() # 성별별 생존자 수
    total = titanic[titanic.survived == 1].survived.count()

    print(f'{gender} : {alive/total : .2f}')


# 객실 등급별 생존자 숫자, 비율
#    1등급 객실 생존자 비율 = (1등급 객실 생존자 숫자) / (전체 생존자 숫자)

print(""); print('- 객실 등급별 생존율')
for grade in titanic.pclass.unique():
    a = titanic[(titanic.survived == 1) & (titanic.pclass == grade)].survived.count() # 등급별 생존자 수
    t = titanic[titanic.survived == 1].survived.count()

    print(f'{grade} : {a/t : .2f}')


print("") ; print('- 10세 미만 승객 숫자/생존자/생존자 비율')
# 10대 미만 승객 숫자
a_10 = len(titanic[titanic.age < 10])
print(f'10세 미만 승객수 : {a_10}')
# 10대 미만 생존자 숫자
a_10_a = len(titanic[(titanic.age < 10) &(titanic.survived == 1)])
print(f'10세 미만 생존자 수 : {a_10_a}')
# 10대 미만 승객이 생존자 배율
print(f'10세 미만 생존율 : {a_10_a/a_10 :.2f}')


전체 생존자 수 : 342.00

- 생존자 성비
male :  0.32
female :  0.68

- 객실 등급별 생존율
3 :  0.35
1 :  0.40
2 :  0.25

- 10세 미만 승객 숫자/생존자/생존자 비율
10세 미만 승객수 : 62
10세 미만 생존자 수 : 38
10세 미만 생존율 : 0.61


In [212]:
titanic[(titanic.survived == 1) & (titanic.pclass == 3)].survived.count()

119

In [216]:
titanic[(titanic.survived == 1)].survived.count() # count non-null values

342

In [ ]:
titanic[(titanic.survived == 1)].value_counts() # count occurrences of each unique value

survived  pclass  sex     age    sibsp  parch  fare   embarked  class   who    adult_male  deck  embark_town  alive  alone  age_bin     
1         1       female  24.00  0      0      69.30  C         First   woman  False       B     Cherbourg    yes    True   (15.0, 30.0]    2
          3       male    32.00  0      0      8.05   S         Third   man    True        E     Southampton  yes    True   (30.0, 45.0]    1
          1       female  16.00  0      0      86.50  S         First   woman  False       B     Southampton  yes    True   (15.0, 30.0]    1
                                        1      39.40  S         First   woman  False       D     Southampton  yes    False  (15.0, 30.0]    1
                                               57.98  C         First   woman  False       B     Cherbourg    yes    False  (15.0, 30.0]    1
                                                                                                                                           ..
          3